In [24]:
import json
import w3lib.html
import random
import os
from datetime import datetime
import requests
import string
import calendar
import html
import locale 
from requests_html import HTMLSession
from requests_html import HTML
import validators
import pandas as pd
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [25]:
URL_SEED = 'https://www.centralweb.cl/noticias/nacional/page/1'
XPATH_HREF = "//h3/a/@href"

In [26]:
class Crawler():
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    #Ruta XPATH hacia los enlaces que queremos recopilar
    xpath_href=XPATH_HREF

    #variable que permite listar todos los enlaces recopilado
    url_list = []
    
    def _init_(self, URL_SEED, XPATH_HREF):        
        pass

    
    def get(self,url):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url,headers=header)        
        return response
    
    def parse(self, response):
        self.url_list = []
        urls = response.html.xpath(self.xpath_href)
        for url in urls:
            self.url_list.append(url)


In [27]:
urls=[]

for page in tqdm(range(3)): # revisemos sólo 2 páginas
    crawler = Crawler()
    response = crawler.get('https://www.centralweb.cl/noticias/nacional/page/'+str(page))
    crawler.parse(response)

    
    for url in crawler.url_list:
        urls.append(url)

    crawler.url_list = urls
    
crawler.url_list

  0%|          | 0/3 [00:00<?, ?it/s]

['https://www.centralweb.cl/en-valparaiso-tottus-inaugura-su-primer-local-del-formato-vecino/',
 'https://www.centralweb.cl/ssangyong-new-musso-grand-es-nombrada-mejor-pick-up-2022/',
 'https://www.centralweb.cl/wyndham-hotels-resorts-anuncia-la-alianza-con-grupo-pettra-y-crece-su-participacion-en-el-pais-en-un-156/',
 'https://www.centralweb.cl/mas-de-400-beneficiados-escuela-en-pudahuel-recibe-equipo-de-sanitizacion-de-agua/',
 'https://www.centralweb.cl/startup-chilena-permite-estar-al-dia-en-todas-las-normativas-ambientales-laborales-de-salud-y-seguridad-en-el-trabajo/',
 'https://www.centralweb.cl/pymes-en-linea-abre-convocatoria-con-mas-de-3-000-becas-para-potenciar-las-ventas-digitales-y-los-emprendimientos-de-todo-chile/',
 'https://www.centralweb.cl/exeed-vx-los-secretos-del-tecnologico-buque-insignia-de-la-nueva-marca-que-aterriza-en-chile/',
 'https://www.centralweb.cl/perfeccionamiento-laboral-aumenta-en-mas-de-un-20-la-demanda-de-formacion-online-para-trabajadores/',
 'htt

In [28]:
len(urls)

60

In [6]:
urls[0]

'https://www.centralweb.cl/en-valparaiso-tottus-inaugura-su-primer-local-del-formato-vecino/'

In [7]:
URL_NEWS = urls[0]
XPATH_TITLE = "//h1"
XPATH_CONTENT ="//div[@class='td-post-content tagdiv-type']//p"

In [8]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    query_extract_title = XPATH_TITLE
    query_extract_text =  XPATH_CONTENT

    def _init_(self):
        pass
   
    def get(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers) 
        return response
    
    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    
    #### main function ####
    def parse(self, response):
        
        #URL
        self.news_url = response.url
        
        #TITLE
        self.news_title = response.html.xpath(self.query_extract_title)[0].text
        self.news_title = self.clean_text(self.news_title)
       
        
        #CONTENT
        self.news_text = ""
        
        html_paragraphs = response.html.xpath(self.query_extract_text)
        
        for paragraph in html_paragraphs:
            paragraph_content = w3lib.html.remove_tags(paragraph.text)    
            text_cleaned = self.clean_text(paragraph_content)
            self.news_text = self.news_text+text_cleaned


In [9]:
scraper = Scraper()
response =scraper.get(URL_NEWS)

In [10]:
scraper.parse(response)

In [11]:
scraper.news_title

'En Valparaíso Tottus inaugura su primer local del formato “Vecino”'

In [12]:
scraper.news_text

'Tottus abrió su primer local en la comuna de Valparaíso y en un nuevo formato: “Tottus Vecino”. Se trata de una experiencia de compra más rápida y eficiente, bajo el concepto “One Stop Shopping”, que ofrece más de 2.300 productos que son parte de la compra cotidiana de los porteños. Este desarrollo va en línea con la estrategia de Territorio de la compañía y continua con la expansión en regiones de Tottus. Se trata del local número 73 y está ubicado en Avenida Francia 748.El gerente general de Tottus, Alejandro Delgado, señaló que “este nuevo formato Vecino nos permitirá una mayor flexibilidad y eficiencia en el servicio al cliente y brindará una atención más cercana a la comunidad de Valparaíso, ofreciendo productos de calidad a precios muy convenientes. Con este esfuerzo queremos impactar de forma positiva en el abastecimiento de la zona, y agradecemos la oportunidad de contribuir al territorio, estamos muy entusiasmados de ser parte activa de la historia del barrio”.El ejecutivo ag

In [13]:
#Verificamos que el scraper devuelve algo
assert(len(scraper.news_title)>0)
assert(len(scraper.news_text)>0)

In [14]:
df = pd.DataFrame(data={'url': [URL_NEWS],'title':[scraper.news_title], 'content':[scraper.news_text]})
df

,url,title,content
0,https://www.centralweb.cl/en-valparaiso-tottus...,En Valparaíso Tottus inaugura su primer local ...,Tottus abrió su primer local en la comuna de V...


In [15]:
XPATH_TITLE = "//h1"
XPATH_CONTENT ="//div[@class='td-post-content tagdiv-type']//p"

for url in tqdm(urls):
    URL_NEWS = url
    scraper = Scraper()
    response =scraper.get(URL_NEWS)
    scraper.parse(response)
    df1 = pd.DataFrame(data={'url': url, 'title':scraper.news_title, 'content':scraper.news_text},index=['url', 'title', 'content'])
    df = df.append(df1, ignore_index=True)
df = df.drop_duplicates()


  0%|          | 0/60 [00:00<?, ?it/s]

In [16]:
df

,url,title,content
0,https://www.centralweb.cl/en-valparaiso-tottus...,En Valparaíso Tottus inaugura su primer local ...,Tottus abrió su primer local en la comuna de V...
4,https://www.centralweb.cl/ssangyong-new-musso-...,SsangYong New Musso Grand es nombrada mejor Pi...,Continúan las buenas noticias para SsangYong M...
7,https://www.centralweb.cl/wyndham-hotels-resor...,Wyndham Hotels & Resorts anuncia la alianza co...,"Wyndham Hotels & Resorts, la compañía de franq..."
10,https://www.centralweb.cl/mas-de-400-beneficia...,Más de 400 beneficiados: Escuela en Pudahuel r...,Con la presencia de autoridades locales y de l...
13,https://www.centralweb.cl/startup-chilena-perm...,Startup chilena permite estar al día en todas ...,"Las normativas ambientales, laborales, de salu..."
16,https://www.centralweb.cl/pymes-en-linea-abre-...,“Pymes en Línea” abre convocatoria con más de ...,Con el objetivo de entregarle a los emprendedo...
19,https://www.centralweb.cl/exeed-vx-los-secreto...,EXEED VX: Los secretos del tecnológico buque i...,Durante el primer semestre desembarcará oficia...
22,https://www.centralweb.cl/perfeccionamiento-la...,Perfeccionamiento laboral: aumenta en más de u...,Cada año son más las empresas que apuestan por...
25,https://www.centralweb.cl/cooperativa-moscin-d...,Cooperativa Moscin de Guarilihue apuesta por v...,Después de una temporada que les reportó la pr...
28,https://www.centralweb.cl/contratacion-interna...,Contratación Internacional en Chile: un mercad...,El mercado chileno cuenta con talentos que res...
